In [3]:
# This code is for process PCAWG ICGC patient coverage files. The output files will be merged into tissue type coverage for MutSigCV
# Author: Yiyun Rao
# Date last edit:05222020

from optparse import OptionParser
import os
from os import listdir
from os.path import isfile, join
import re
import time
from shutil import copyfile

In [4]:
# Error Area
class TumorTypeNotFoundError(Exception):
    def __init__(self, *args):
        if args:
            self.message = args[0]
        else:
            self.message = None

    def __str__(self):
        if self.message:
            return 'TumorTypeNotFoundError, {0} '.format(self.message)
        else:
            return 'TumorTypeNotFoundError has been raised'


class PathAlreadyExistsError(Exception):
    def __init__(self, *args):
        if args:
            self.message = args[0]
        else:
            self.message = None

    def __str__(self):
        if self.message:
            return 'PathAlreadyExists, {0} '.format(self.message)
        else:
            return 'PathAlreadyExists has been raised'

In [7]:
def get_tumor_type(info_f):
    dict_patient_tumor = {}
    list_tumor_type =[]
    with open(info_f,'r') as f_info:
        next(f_info)
        for lines in f_info:
            line_split = lines.split("\t")
            tumor_type = line_split[10].split('-')[0]
            tumor_id = line_split[0]
            if tumor_id not in dict_patient_tumor:
                dict_patient_tumor[tumor_id] = tumor_type
            if tumor_type not in list_tumor_type:
                list_tumor_type.append(tumor_type)
    
    return dict_patient_tumor,list_tumor_type
patient_tumor_dict, tumor_type_list = get_tumor_type('../anno_ref/PCAWG_sample_info.txt')

In [9]:
cwd = '../cov_out'
new_path = cwd + "/organ_type_coverage"
if not os.path.exists(new_path):
    os.makedirs(new_path)
else:
    raise PathAlreadyExistsError("DIRECTORY ALREADY EXISTS")

for t in tumor_type_list:
    os.makedirs(new_path+'/'+t)

In [10]:
coverage_path = '../cov_out/coverage_complete_output'
list_coverage = listdir(coverage_path)
for f in list_coverage:
    patient_name = f.split('.')[0]
    try:
        type_tumor = patient_tumor_dict[patient_name]
        f_path_old = join(coverage_path,f)
        f_path_new = join(new_path,type_tumor,f)
        copyfile(f_path_old, f_path_new)
    except KeyError:
        print f